## Preprocess

In [1]:
import numpy as np 
import pandas as pd

In [2]:
x_transaction = pd.read_csv('../../kkbox-churn-prediction-challenge/50_under_sample/train_50_pct_undersampled_transactions.csv')

In [3]:
x_transaction.head(1)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,P1i0ZUBXvfn2sb0f8jcIbMqTxtY5esoMhYDtarLoW9o=,32,410,1788,1788,0,2017-03-20T00:00:00,2018-05-07T00:00:00,0


In [4]:
x_transaction.shape

(1032715, 9)

In [5]:
x_transaction['payment_plan_days'].unique()

array([410, 195, 180,  60, 200, 100,  90, 400, 395,  70, 360,  30, 270,
         0, 120, 450,   7, 365,  99, 240,  10,  21,  14,  66,   2, 110,
        80,   3, 415, 230,   1,  31,  15,  35])

### Variables generated from plan_list_price & actual_amount_paid
1. discount_amount = plan_list_price - actual_amount_paid
There is negative discount_amount, therefore, check each of the situation, get the following:

7 days: 35

30 days: 149/129/119/150/100/134/131

60 days: 300

80 days: 400

90 days: 480/447/450

180 days = 894/930/799/536

365 days = 1788

2. Current method:

a) If plan_list_price = 0, plan_list_price=actual_amount_paid and assgin payment_plan_days by checking actual_amount_paid

b) If actual_amount_paid=0, get a new column called free

c) Discount = plan_list_price - actual_amount_paid

3. Problem of this method

The negative list may be more with the amount of data incereasing. But seems inappropriate to delete them because it usually stands abnormal situations.

In [6]:
# get the discount_amount
x_transaction['discount_amount'] = x_transaction['plan_list_price'] - x_transaction['actual_amount_paid']

In [ ]:
x_transaction['discount_amount'].unique()

In [ ]:
# the negative ones
len(x_transaction[x_transaction['discount_amount']<0])

In [ ]:
x_transaction[x_transaction['discount_amount']==-500]

In [ ]:
# change "plan_list_price"
x_transaction['plan_list_price'] = np.where(x_transaction['discount_amount']<0, x_transaction['actual_amount_paid'],x_transaction['plan_list_price'])

In [ ]:
# change "payment_plan_days"
conditions = [
    (x_transaction['discount_amount'] ==-35),
    (x_transaction['discount_amount'].isin([-149,-129,-119,-150,-100,-134,-131])),
    (x_transaction['discount_amount'] ==-300),
    (x_transaction['discount_amount'] ==-400),
    (x_transaction['discount_amount'].isin([-480,-447,-450])),
    (x_transaction['discount_amount'].isin([-894,-930,-799,-536])),
    (x_transaction['discount_amount'] ==-1788)]
choices = [7, 30, 60,80,90,180,365]
x_transaction['payment_plan_days'] = np.select(conditions,choices,x_transaction['payment_plan_days'])

In [ ]:
(x_transaction[x_transaction['discount_amount']<0].payment_plan_days>0).unique()

In [ ]:
# change the negative discount to be 0
x_transaction['discount_amount'] = np.where(x_transaction['discount_amount']<0, 0, x_transaction['discount_amount'])

In [7]:
# get a new column called free
x_transaction['free']=0
x_transaction['free'] = np.where(x_transaction['actual_amount_paid']==0, 1, x_transaction['free'])

### Variables related to date

In [8]:
x_transaction=x_transaction.sort_values(by=['msno', 'transaction_date'])
x_transaction.head(1)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount_amount,free
656208,++9JUsaZioofS6Fb20Z0z2QOlWNzakO0PRF5GZ75yi4=,37,31,149,149,1,2015-01-09T00:00:00,2015-02-10T00:00:00,0,0,0


In [9]:
# days between transaction & expiration
x_transaction.membership_expire_date=pd.to_datetime(x_transaction.membership_expire_date)
x_transaction.transaction_date=pd.to_datetime(x_transaction.transaction_date)
x_transaction['length']=(x_transaction['membership_expire_date']-x_transaction['transaction_date']).dt.days

In [10]:
x_transaction.head(2)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,discount_amount,free,length
656208,++9JUsaZioofS6Fb20Z0z2QOlWNzakO0PRF5GZ75yi4=,37,31,149,149,1,2015-01-09,2015-02-10,0,0,0,32
393797,++9JUsaZioofS6Fb20Z0z2QOlWNzakO0PRF5GZ75yi4=,37,31,149,149,1,2015-02-09,2015-03-10,0,0,0,29


In [11]:
# the average amount paid by user in each payment
x_transaction['amtperday']=x_transaction['actual_amount_paid']/x_transaction['payment_plan_days']

### Sort the table to get information from records between each trasanction for a user

In [38]:
# get the consecutive difference from rows
x_transaction["d_transaction"] = x_transaction["transaction_date"].diff(1).fillna(0).astype('timedelta64[D]').astype(int)
x_transaction.d_transaction = np.where(x_transaction.d_transaction <= 0, np.NaN,x_transaction.d_transaction)

In [41]:
# same for expire_date
x_transaction["d_expire_date"] = x_transaction["membership_expire_date"].diff(1).fillna(0).astype('timedelta64[D]').astype(int)
x_transaction.d_expire_date = np.where(x_transaction.d_expire_date <= 0, np.NaN,x_transaction.d_expire_date)

### A potential sign of churn (auto_renew=0 & expiration in 201702)

In [14]:
from datetime import datetime
startdate = datetime(2017, 2, 1)
enddate = datetime(2017, 2, 28)
x_transaction['potential_churn'] = np.where((x_transaction['is_auto_renew']==0)&(x_transaction['membership_expire_date']>=startdate) 
                                            & (x_transaction['membership_expire_date'] <= enddate), 1,0)

In [ ]:
x_transaction.head(1)

### Groupby Customer and generate related variables

In [15]:
# define some common used funcitons
most_common = lambda x: pd.Series.mode(x)[0]
most_common.__name__ = 'most_common'
max_min_diff = lambda x: x.max() - x.min()
max_min_diff.__name__ = 'max_min_diff'
def pcet_of_zero(x):
    return 1-(x.mean())
def change_or_not(x):
    return (x.nunique()-1)
def find_positive_pct(x):
    return ((x>0).sum()/x.count())
def has_discount(x):
    return x.nunique()>1

In [16]:
# def change column name
def chagne_name(df):
    table = df.columns.levels[0][0]+"_"
    cols=df.columns.levels[1]
    cols=cols[-1:]+cols[:-1]
    cols=["{}".format(table) + s for s in cols]
    cols.insert(0, "msno")
    return cols

# def change column name 2:
def change_name_2(df):
    df.columns = ["_".join(x) for x in df.columns.ravel()]
    df.rename(columns={'msno_':'msno'}, inplace=True)
    return df

In [17]:
# payment_method
payment_method=x_transaction.groupby('msno',as_index=False).agg({
    'payment_method_id':['nunique',most_common]})
payment_method.columns=chagne_name(payment_method)

In [ ]:
payment_method.head(1)

In [18]:
# payment_plan_days
payment_plan_days=x_transaction.groupby('msno',as_index=False).agg({
    'payment_plan_days':['nunique',most_common]})
payment_plan_days.columns=chagne_name(payment_plan_days)

In [ ]:
payment_plan_days.head(1)

In [19]:
# plan_list_price
plan_list_price=x_transaction.groupby('msno',as_index=False).agg({
    'plan_list_price':['nunique', most_common,'mean',max_min_diff]})
plan_list_price.columns=chagne_name(plan_list_price)

In [ ]:
plan_list_price.head(1)

In [20]:
# actual_amount_paid
actual_amount_paid=x_transaction.groupby('msno',as_index=False).agg({
    'actual_amount_paid':['nunique', most_common,'mean',max_min_diff]})
actual_amount_paid.columns=chagne_name(actual_amount_paid)

In [ ]:
actual_amount_paid.head(1)

In [21]:
# is_auto_renew
is_auto_renew=x_transaction.groupby('msno',as_index=False).agg({
    'is_auto_renew':[pcet_of_zero, change_or_not,most_common]})
is_auto_renew.columns=chagne_name(is_auto_renew)

In [22]:
# is_cancel
is_cancel=x_transaction.groupby('msno',as_index=False).agg({
    'is_cancel':['mean', change_or_not]})
is_cancel.columns=chagne_name(is_cancel)

In [ ]:
is_cancel.head(1)

In [23]:
# discount_amount
discount_amount=x_transaction.groupby('msno',as_index=False).agg({
    'discount_amount':[find_positive_pct, 'mean','sum',has_discount]})
discount_amount.columns=chagne_name(discount_amount)

In [ ]:
discount_amount.head(1)

In [24]:
# free
free=x_transaction.groupby('msno',as_index=False).agg({
    'free':['mean',has_discount]})
free.columns=chagne_name(free)

In [ ]:
free.columns

In [25]:
# length
length=x_transaction.groupby('msno',as_index=False).agg({
    'length':['mean','sum','std','first']}).fillna(0)

In [26]:
# another rename method
length=change_name_2(length)

In [ ]:
length.head(1)

In [27]:
# amtperday
amtperday=x_transaction.groupby('msno',as_index=False).agg({
    'amtperday':['mean']}).fillna(0)
amtperday=change_name_2(amtperday)

In [ ]:
amtperday.head(1)

In [28]:
# transaction_date
transaction_date=x_transaction.groupby('msno',as_index=False).agg({
    'transaction_date':['first']})
transaction_date=change_name_2(transaction_date)

In [40]:
# d_transaction
d_transaction=x_transaction.groupby('msno',as_index=False).agg({
    'd_transaction':['mean','std']}).fillna(0)
d_transaction=change_name_2(d_transaction)

In [ ]:
d_transaction.head(1)

In [42]:
# d_expire_date
d_expire_date=x_transaction.groupby('msno',as_index=False).agg({
    'd_expire_date':['mean','std','sum']}).fillna(0)
d_expire_date=change_name_2(d_expire_date)

In [ ]:
d_expire_date.head(1)

In [43]:
# potential_churn
potential_churn=x_transaction.groupby('msno',as_index=False).agg({
    'potential_churn':[has_discount]})
potential_churn.columns=chagne_name(potential_churn)

In [ ]:
potential_churn.head(1)

In [44]:
data_frames=[payment_method,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,discount_amount,free,length,amtperday,transaction_date,d_transaction,d_expire_date,potential_churn]

In [45]:
from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['msno'],
                                            how='inner'), data_frames)

In [46]:
df_merged['membership_days']=df_merged['d_expire_date_sum']+df_merged['length_first'] 
# get total membership days
df_merged=df_merged.drop(['d_expire_date_sum', 'length_first'], axis=1)

In [ ]:
df_merged.to_csv('../../kkbox-churn-prediction-challenge/train/X_train_transactions.csv',index=False)

In [ ]:
df = pd.read_csv('../../kkbox-churn-prediction-challenge/train/X_train_transactions.csv')

In [ ]:
df.shape